In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import chemiscope
from widget_code_input import WidgetCodeInput
from ipywidgets import Layout, Output, Text, HTML, HBox, VBox
from scwidgets import (AnswerRegistry, TextareaAnswer, CodeDemo,
                       ParametersBox, PyplotOutput, ClearedOutput,
                       AnimationOutput,CheckRegistry,Answer)
from collections.abc import Iterable

import ase
from ase.io import read, write
from ase.calculators import lj, eam
from ase.optimize import BFGS, LBFGS
from tqdm.notebook import tqdm
import math
import matplotlib
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

from warnings import filterwarnings
filterwarnings("ignore", message="chemiscope")

matplotlib.rcParams["animation.embed_limit"] = 50

In [ ]:
%%html
<style>
.jp-CodeCell.jp-mod-outputsScrolled .jp-Cell-outputArea  {  height:auto !important;
    max-height: 5000px; overflow-y: hidden }
</style>
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:4000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [ ]:
check_registry = CheckRegistry() 
answer_registry = AnswerRegistry(prefix="module_06")
display(answer_registry)

You can write here general comments you may have on this module. 

In [ ]:
module_summary = TextareaAnswer("general comments on this module")
answer_registry.register_answer_widget("module-summary", module_summary)
display(module_summary)

_Reference textbook / figure credits: Allen, Tildesley, Computer simulations of liquids (2017): Chapter 3_

# Classical mechanics: Newton, Hamilton and planetary motion

The basic idea behind molecular dynamics is to apply to atoms the same kinematic laws that are applied to macroscopic objects. This is as simple as Newton's second law: if $\mathbf{x}$ corresponds to the position of a particle, its motion is governed by 

$$
\ddot{\mathbf{x}} = \mathbf{f}/m = -\frac{1}{m}\frac{\partial V}{\partial \mathbf{x}} 
$$

where we also use the definition of the force acting on each particle as the derivative of a potential energy. 

The equations of motion can also be written in several alternative forms. A particularly elegant one involves formulating _Hamilton's equations_

$$
\dot{\mathbf{x}} = \mathbf{p}/m; \quad \dot{\mathbf{p}} = \mathbf{f}
$$

where $\mathbf{p}$ is the _momentum_ of the particle. It's clear that the second law can be recovered by taking the time derivative of the first equation, and substituting the equation for the derivative of the momentum. 
Note also that these equations can be written in a symmetric form by defining the Hamiltonian 

$$
H(\mathbf{x},\mathbf{p}) = V(\mathbf{x}) + \mathbf{p}^2/2m
$$

that corresponds to the total (potential+kinetic) energy. Then, one can write

$$
\dot{\mathbf{x}} = \frac{\partial H}{\partial\mathbf{p}}; \quad \dot{\mathbf{p}} = -\frac{\partial H}{\partial\mathbf{x}}.
$$

<span style="color:blue">**01** Compute $d{H}/d{t}$ for a particle that follows Hamilton's equations. Sketch the key steps of the derivation. If at $t=0$ the Hamiltonian evaluates to $1$kJ, how large will it be at $t=10s$? </span>

In [ ]:
ex1_txt = TextareaAnswer("Enter your answer here")
answer_registry.register_answer_widget("ex1-answer", ex1_txt)
display(ex1_txt)

The widget below shows the analytical solution for the motion of two planets with a given mass and initial velocities. The input is the initial position and velocity of the first body. The initial position and velocity of the other one are set such a way to make the center of the mass motionless and located at [0, 0] 
The initial parameters are set to match roughly those for the earth ($m_2$) and moon ($m_1$). Experiment with different parameters.

In [ ]:
# constants to be used for units conversion
G = 6.67430e-11 # gravitational constant in SI units
m_earth = 5.972e24 # mass of earth in kg
d_em = 383.4e6 # earth-moon distance in m

def get_theoretical_trajectories(x1_initial, x2_initial, v1_initial, v2_initial, 
                                 m1, m2, k=G, n_points = 200):
    """Computes the elliptical orbits for the Keplerian 2-body problem. 
    k is the universal gravitational constant. Assumes inputs in SI units"""
        
    delta_x_initial = x1_initial - x2_initial
    delta_v_initial = v1_initial - v2_initial
    mu = m1 * m2 / (m1 + m2)
    r = np.sqrt(np.sum(delta_x_initial * delta_x_initial))
    L = mu * np.abs(np.cross(delta_x_initial, delta_v_initial))
    alpha = k*m1*m2
    E = -alpha / r + 0.5 * mu * np.sum(delta_v_initial ** 2)
    e = np.sqrt(1 + 2 * E * L * L / (alpha**2 * mu))
    theta_initial = np.arctan2(delta_x_initial[1], delta_x_initial[0])
    
    if np.abs(e) > 1e-7:
        cos = (L * L / (alpha * mu * r) - 1) / e
        # fixes rounding errors
        cos = min(1,max(-1,cos))
        
        delta_theta = np.arccos(cos)
        first_theta_0 = theta_initial - delta_theta
        second_theta_0 = theta_initial + delta_theta

        def get_velocity_direction(theta_0):
            r_p_theta = -L * L / (alpha * mu) * e * np.sin(theta_0 - theta_initial) / (e * np.cos(theta_0 - theta_initial) + 1) ** 2
            vx = r_p_theta * np.cos(theta_initial) - np.sin(theta_initial)
            vy = r_p_theta * np.sin(theta_initial) + np.cos(theta_initial)

            length = np.sqrt(vx ** 2 + vy ** 2)
            return  np.array([vx / length, vy / length])

        first_direction = get_velocity_direction(first_theta_0)
        second_direction = get_velocity_direction(second_theta_0)

        if np.abs(np.dot(first_direction, delta_v_initial)) > np.abs(np.dot(second_direction, delta_v_initial)):
            theta_0 = first_theta_0
        else:
            theta_0 = second_theta_0
    else:
        theta_0 = 0.0
    
    theoretical_1, theoretical_2 = [], []
    has_break = False
    for theta in np.linspace(0, 2 * np.pi, n_points):
        r_now = L * L / (alpha * mu * (1 + e * np.cos(theta - theta_0)))
        if (r_now > 0.0):
            r_first = r_now * m2 / (m1 + m2)
            r_second = r_now * m1 / (m1 + m2)

            theoretical_1.append([r_first * np.cos(theta), r_first * np.sin(theta)])
            theoretical_2.append([-r_second * np.cos(theta), -r_second * np.sin(theta)])
        else:
            theoretical_1.append(None)
            theoretical_2.append(None)
            has_break = True
            
            
    if not has_break:
        theoretical_1_aligned = np.array(theoretical_1)
        theoretical_2_aligned = np.array(theoretical_2)
    else:
        if (theoretical_1[0] is None) and (theoretical_1[-1] is None):
            theoretical_1_aligned, theoretical_2_aligned = [], []
            for i in range(len(theoretical_1)):
                if theoretical_1[i] is not None:
                    theoretical_1_aligned.append(theoretical_1[i])
                    theoretical_2_aligned.append(theoretical_2[i])
            theoretical_1_aligned = np.array(theoretical_1_aligned)
            theoretical_2_aligned = np.array(theoretical_2_aligned)
        else:
            theoretical_1_aligned, theoretical_2_aligned = [], []
            for i in range(len(theoretical_1)):
                if theoretical_1[i] is None:
                    if (i + 1 == len(theoretical_1)) or (theoretical_1[i + 1] is not None):
                        begining = i + 1
                        break
            for i in range(begining, len(theoretical_1)):
                theoretical_1_aligned.append(theoretical_1[i])
                theoretical_2_aligned.append(theoretical_2[i])
            for i in range(len(theoretical_1)):
                if theoretical_1[i] is None:
                    break
                theoretical_1_aligned.append(theoretical_1[i])
                theoretical_2_aligned.append(theoretical_2[i])
            theoretical_1_aligned = np.array(theoretical_1_aligned)
        theoretical_2_aligned = np.array(theoretical_2_aligned)

    return theoretical_1_aligned, theoretical_2_aligned, E, e

In [ ]:
def plot_theoretical(m1, m2, x1, y1, v1_x, v1_y, visualizers, k=G,
                     n_points = 200, margin = 0.1):
    """ Plots the theoretical gravitational 2B problem trajectory. 
    masses are expressed in earth masses, positions in units of the earth-moon distance
    and velocities in km/s (which is roughly the mean orbital velocity of the moon)
    """
    
    ax = visualizers[1].figure.get_axes()[0]
    # converts to SI units
    m1 *= m_earth; m2*= m_earth
    x1 *= d_em; y1 *= d_em
    v1_x *= 1e3
    v1_y *= 1e3
    
    x1_initial = np.array([x1, y1])
    v1_initial = np.array([v1_x, v1_y])
    if np.abs(np.cross(x1_initial, v1_initial)) < 1e-7:
        ax.set_title(f"angular momentum is zero, cannot evaluate analytical solution")
        return
    
    x2_initial = -m1 * x1_initial / m2
    v2_initial = -m1 * v1_initial / m2
        
    theoretical_1, theoretical_2, E, e = get_theoretical_trajectories(x1_initial, x2_initial, v1_initial, v2_initial, 
                                 m1, m2, k, n_points = n_points)
    
    if (e < 1):
        min_x = min(np.min(theoretical_1[:, 0]), np.min(theoretical_2[:, 0]))
        max_x = max(np.max(theoretical_1[:, 0]), np.max(theoretical_2[:, 0]))
        spread = max_x - min_x
        min_x -= spread * margin
        max_x += spread * margin

        min_y = min(np.min(theoretical_1[:, 1]), np.min(theoretical_2[:, 1]))
        max_y = max(np.max(theoretical_1[:, 1]), np.max(theoretical_2[:, 1]))
        spread = max_y - min_y
        min_y -= spread * margin
        max_y += spread * margin
    else:
        first_d = np.min(np.sqrt(np.sum(theoretical_1 ** 2, axis = 1)))
        second_d = np.min(np.sqrt(np.sum(theoretical_2 ** 2, axis = 1)))
        first_d_initial = np.sqrt(np.sum(x1_initial ** 2))
        second_d_initial = np.sqrt(np.sum(x2_initial ** 2))
        
        limit = 5 * max(first_d, second_d, first_d_initial, second_d_initial)
        min_x, min_y = -limit, -limit
        max_x, max_y = limit, limit
        
   
    min_both = min(min_x, min_y)
    max_both = max(max_x, max_y)
    ax.set_xlim([min_both, max_both])
    ax.set_ylim([min_both, max_both])
    
    max_spread = max(max_x - min_x, max_y - min_y)
    
    circle1 = plt.Circle((x1_initial[0], x1_initial[1]), 0.03 * max_spread, color='b',
                             label = f'first; $m$ = {m1/m_earth} $m_E$')
    ax.add_patch(circle1)
    
    
    circle2 = plt.Circle((x2_initial[0], x2_initial[1]), 0.03 * max_spread, color='g', 
                             label = f'second; $m$ = {m2/m_earth} $m_E$')
    ax.add_patch(circle2)
    ax.plot(theoretical_1[:, 0], theoretical_1[:, 1], color = 'b')
    ax.plot(theoretical_2[:, 0], theoretical_2[:, 1], color = 'g')
    
    
    def get_center_mass():
        return (x1_initial * m1 + x2_initial * m2) / (m1 + m2)
    center_mass = get_center_mass()
    
    ax.plot([center_mass[0]], [center_mass[1]], 'x', color = 'red',
                   label = 'center of mass')
    
    first_length = np.sqrt(np.sum(v1_initial ** 2))
    second_length = np.sqrt(np.sum(v2_initial ** 2))
    
    max_length = max(first_length, second_length)
   
    multiplier = 0.3 * max_spread / max_length
    
    ax.arrow(x1_initial[0], x1_initial[1], v1_initial[0] * multiplier , v1_initial[1] * multiplier, fc = 'b',
             head_width = 0.03 * max_spread)
    ax.arrow(x2_initial[0], x2_initial[1], v2_initial[0] * multiplier , v2_initial[1] * multiplier, fc = 'g',
             head_width = 0.03 * max_spread)
    
    ax.legend(loc = 'upper left')
    if e < 1.0:
        ax.set_title(f"Energy = {(E/1e30):9.3f} $\\times 10^{{30}}$J, eccentricity = {e:9.3f}")
    else:
        ax.set_title(f"Energy = {(E/1e30):9.3f} $\\times 10^{{30}}$J, eccentricity = {e:9.3f}\n trajectory is unbound")
                         
    
fig_ax_analytical, _ = plt.subplots(figsize=(6, 6))
analytical_pyplot_output = PyplotOutput(fig_ax_analytical)
analytical_pb = ParametersBox( m1 = (0.01, 0.01, 10, 0.01, r'$m_1$ / $m_\mathrm{Moon}$'),
                                       m2 = (1.0, 0.01, 10, 0.01, r'$m_2$ / $m_\mathrm{Earth}$'),
                                       x1 = (1.0, -10, 10, 0.01, r'$x_1(0)$ / $d(\mathrm{earth-moon})$'),
                                       y1 = (0.0, -10, 10, 0.01, r'$y_1(0)$ / $d(\mathrm{earth-moon})$'),
                                       v1_x = (0.0, -2, 2, 0.01, r"$v_{1x}$ / km/s"), # r'$v_{x1}(0)$ / km/s'),
                                       v1_y = (1.0, -2, 2, 0.01, r'$v_{1y}$ / km/s'),
                                       )

analytical_code_demo = CodeDemo(
            input_parameters_box=analytical_pb,
            visualizers = [ClearedOutput(),analytical_pyplot_output],
            update_visualizers = plot_theoretical)

In [ ]:
display(analytical_code_demo)
analytical_code_demo.run_demo()

# Integrators: numerically solving the equations of motion

Even in the simple case of a central potential, analytical solutions are possible only for a two-body setup. Even just the [three-body case](https://en.wikipedia.org/wiki/Three-body_problem) (e.g. a Sun-Earth-Moon scenario) doesn't have a closed-form solution. 

It then becomes necessary to use numerical methods to integrate the equations of motion. The simplest method possible corresponds to a naïve discretization of the expression for the first-order derivatives in the Hamiltonian formulation:

$$
\mathbf{x}(dt) = \mathbf{x}(0) + dt\  \mathbf{p}(0)/m\\
\mathbf{p}(dt) = \mathbf{p}(0) + dt\ \mathbf{f}(0)\\
$$

This is called _forward Euler_ method, and $dt$ is the _timestep_ used to integrate the equations of motion. The procedure is iterated many times, until the trajectory has evolved for a sufficiently long time to 

<span style="color:blue">**02a** Complete the function below to implement a forward Euler integrator for the equations of motion of two planets, given the masses and the initial condition. </span>

_Most of the routine is already implemented, only modify the part indicated, but try to understand what the rest of the code is doing. The routine returns the time series of planet positions, potential and kinetic energy._

In [ ]:
def simulate(updater, m1, m2, x1_initial, x2_initial, v1_initial, v2_initial, dt, num):
    x1, x2, v1, v2 = x1_initial, x2_initial, v1_initial, v2_initial
    x1_log, x2_log, v1_log, v2_log = [x1], [x2], [v1], [v2]
    for _ in tqdm(range(num), leave=True):
        x1, x2, v1, v2 = updater(m1, m2, x1, x2, v1, v2, dt)
        
        x1_log.append(x1)
        x2_log.append(x2)

        v1_log.append(v1)
        v2_log.append(v2)

    return np.array(x1_log), np.array(x2_log), np.array(v1_log), np.array(v2_log)
        
def simulate_center_fix(updater, m1, m2, x1_initial, v1_initial, dt, T):
    
    x2_initial = -m1 * x1_initial / m2
    v2_initial = -m1 * v1_initial / m2
    num = math.ceil(T / dt)
    return simulate(updater, m1, m2, x1_initial, x2_initial, v1_initial, v2_initial, dt, num)

def make_anim_simulation(updater, base_figure, ax, m1, m2, x1, y1, v1_x, v1_y, dt, T, 
                         duration = 10, fps = 15, margin = 0.1, compress_to = 10000):
    
    """ Creates a matplotlib animation object visualizing a trajectory of a planetary body, 
    and returns it. """ 
    
    m1 *= m_earth; m2*= m_earth
    x1 *= d_em; y1 *= d_em
    v1_x *= 1e3
    v1_y *= 1e3
    dt*=86400; T*=86400
   
    x1_initial = np.array([x1, y1])
    v1_initial = np.array([v1_x, v1_y])
    if np.abs(np.cross(x1_initial, v1_initial)) < 1e-7:
        print(f"angular momentum is zero \n It is inappropriate initial condition for this model")
        return
    
    print("Performing simulation...")
    x1_log, x2_log, v1_log, v2_log = simulate_center_fix(updater, m1, m2, x1_initial, v1_initial, dt, T)
        
    print("Simulation finished.\n")
    
    print("Making animation...")
    
    delta_x_initial = x1_log[0] - x2_log[0]
    delta_v_initial = v1_log[0] - v2_log[0]

    theoretical_1, theoretical_2, E, e = get_theoretical_trajectories(x1_log[0], x2_log[0], v1_log[0], v2_log[0], 
                                                                    m1, m2, G)

    T = dt * x1_log.shape[0]
    
    total_draws = duration * fps    
    if compress_to is not None:
        step = math.floor(x1_log.shape[0] / compress_to)
        if step > 1:
            x1_log = x1_log[::step]
            x2_log = x2_log[::step]
            v1_log = v1_log[::step]
            v2_log = v2_log[::step]
            dt = dt * step

    def get_energies():
        deltas = x1_log - x2_log
        distances = np.sqrt(np.sum(deltas ** 2, axis = 1))
        potential_energies = -G*m1*m2 / distances
        kinetic_energies = 0.5 * (m1 * np.sum(v1_log ** 2, axis = 1) + m2 * np.sum(v2_log ** 2, axis = 1))
        return potential_energies, kinetic_energies, potential_energies + kinetic_energies

    def get_center_mass():
        return (x1_log * m1 + x2_log * m2) / (m1 + m2)

    potential_energies, kinetic_energies, energies = get_energies()
    center_mass = get_center_mass()
    
    if e < 1:
        min_x = min(np.min(x1_log[:, 0]), np.min(x2_log[:, 0]), np.min(theoretical_1[:, 0]), np.min(theoretical_2[:, 0]))
        max_x = max(np.max(x1_log[:, 0]), np.max(x2_log[:, 0]), np.max(theoretical_1[:, 0]), np.max(theoretical_2[:, 0]))
    else:
        min_x = min(np.min(x1_log[:, 0]), np.min(x2_log[:, 0]))
        max_x = max(np.max(x1_log[:, 0]), np.max(x2_log[:, 0]))
    
    spread = max_x - min_x
    min_x -= spread * margin
    max_x += spread * margin
    
    if e < 1:
        min_y = min(np.min(x1_log[:, 1]), np.min(x2_log[:, 1]), np.min(theoretical_1[:, 1]), np.min(theoretical_2[:, 1]))
        max_y = max(np.max(x1_log[:, 1]), np.max(x2_log[:, 1]), np.max(theoretical_1[:, 1]), np.max(theoretical_2[:, 1]))
    else:
        min_y = min(np.min(x1_log[:, 1]), np.min(x2_log[:, 1]))
        max_y = max(np.max(x1_log[:, 1]), np.max(x2_log[:, 1]))
        
    spread = max_y - min_y
    min_y -= spread * margin
    max_y += spread * margin

    min_both = min(min_x, min_y)
    max_both = max(max_x, max_y)
    max_spread = max(max_x - min_x, max_y - min_y)
    
    min_e = min(np.min(potential_energies), np.min(kinetic_energies), np.min(energies))
    max_e = max(np.max(potential_energies), np.max(kinetic_energies), np.max(energies))
    delta = max_e - min_e
    min_e -= delta * margin
    max_e += delta * margin

    def init():

        ax[0].set_xlabel('$x$/m', fontsize = 12)
        ax[0].set_ylabel('$y$/m', fontsize = 12)
        ax[0].set_xlim([min_both, max_both])
        ax[0].set_ylim([min_both, max_both])


        line_center, = ax[0].plot([], 'x', color = 'red',
                       label = 'center of mass')

        line_theoretical_1, = ax[0].plot([], color = 'b', label = 'analytical traj. 1')
        line_theoretical_2, = ax[0].plot([], color = 'g', label = 'analytical traj. 2')

        line_1,  = ax[0].plot([], color = 'gray', alpha = 0.5)
        line_2,  = ax[0].plot([], color = 'gray', alpha = 0.5)

        circle_1 = plt.Circle((x1_log[0, 0], x1_log[0, 1]), 0.03 * max_spread, color='b',
                                 label = f'first; $m$ = {m1/m_earth} $m_E$')
        ax[0].add_patch(circle_1)

        circle_2 = plt.Circle((x2_log[0, 0], x2_log[0, 1]), 0.03 * max_spread, color='g', 
                                 label = f'second; $m$ = {m2/m_earth} $m_E$')
        ax[0].add_patch(circle_2)

        ax[0].legend(loc = 'upper left')
        if e < 1.0:
            ax[0].set_title(f"Energy = {(E/1e30):9.3f} $\\times 10^{{30}}$J, eccentricity = {e:9.3f}")
        else:
            ax[0].set_title(f"Energy = {(E/1e30):9.3f} $\\times 10^{{30}}$J, eccentricity = {e:9.3f}\n trajectory is unbound")


        line_pot, = ax[1].plot([], label = 'potential energy')
        line_kin, = ax[1].plot([], label = 'kinetic energy')
        line_tot, = ax[1].plot([], label = 'total energy')


        ax[1].set_xlim([0, T/86400])
        ax[1].set_ylim([min_e, max_e])
        ax[1].legend(loc = 'upper right')
        ax[1].set_xlabel("Time / days", fontsize = 12)
        ax[1].set_ylabel("Energy / J", fontsize = 12)
        return base_figure, ax, line_pot, line_kin, line_tot, line_theoretical_1,\
            line_theoretical_2, line_center, circle_1, circle_2, line_1, line_2

    base_figure, ax, line_pot, line_kin, line_tot, line_theoretical_1, line_theoretical_2, line_center, circle_1, circle_2, line_1, line_2 = init()

    def animate(frame_num):
        index = int(x1_log.shape[0] * frame_num / total_draws)
        if index > x1_log.shape[0] - 1:
            index = x1_log.shape[0] - 1
        line_pot.set_data((np.arange(index) * dt/86400, potential_energies[0:index]))
        line_kin.set_data((np.arange(index) * dt/86400, kinetic_energies[0:index]))
        line_tot.set_data((np.arange(index) * dt/86400, energies[0:index]))

        line_theoretical_1.set_data((theoretical_1[:, 0], theoretical_1[:, 1]))
        line_theoretical_2.set_data((theoretical_2[:, 0], theoretical_2[:, 1]))
        line_center.set_data((center_mass[index, 0], center_mass[index, 1]))
        circle_1.center = x1_log[index, 0], x1_log[index, 1]
        circle_2.center = x2_log[index, 0], x2_log[index, 1]
        line_1.set_data((x1_log[0:index, 0], x1_log[0:index, 1]))
        line_2.set_data((x2_log[0:index, 0], x2_log[0:index, 1]))
        return line_pot, line_kin, line_tot, line_theoretical_1, line_theoretical_2, line_center, circle_1, circle_2, line_1, line_2

    anim = FuncAnimation(base_figure, animate, frames=tqdm(range(total_draws), leave=True), 
                         interval=duration, blit = True)
    
    anim =  HTML(anim.to_jshtml(fps=fps))
    print("Animation finished.")
    
    return anim

In [ ]:
ex02_wci = WidgetCodeInput(
        function_name="euler_update", 
        function_parameters="m1, m2, r1, r2, v1, v2, dt",
        docstring="""Update the position of the particles in a two-body problem by
        the forward Euler method described above.
        Takes as arguments the masses of the two bodies (m1,m2), the initial positions
        and velocities (r1,r2,v1,v2) and the time step (dt). Returns the new positions
        and velocities (r1_new,r2_new,v1_new,v2_new). 
        >> All positions and velocities are numpy arrays. <<
""",
        function_body="""

import numpy as np
return r1, r2, v1, v2 # <-- remove this line after having finished the implementation of the function
G = 6.67430e-11 # gravitational constant in SI units
# you can compute forces by calling this function with the positions of the two bodies
def get_forces(r1, r2):
    delta = r2 - r1
    norm = np.sqrt(np.sum(delta ** 2))
    direction = delta / norm

    f1 = direction * G*m1*m2 / (norm ** 2)
    f2 = -f1
    return f1, f2

# Modify this to implement a forward Euler integrator. 
r1_new = r1
r2_new = r2

v1_new = v1
v2_new = v2

return r1_new, r2_new, v1_new, v2_new
"""
        )

def convert_celestial_input(m1, m2, r1, r2, v1, v2, dt):
    
    m1 *= m_earth; m2*= m_earth
    r1 *= d_em; r2 *= d_em
    v1 *= 1e3; v2 *= 1e3
    dt*=86400
    
    return (m1, m2, r1, r2, v1, v2, dt)
    
first_input = (0.01, 1.0, np.array([1.0, 0.0]), np.array([-1.0, 0.0]), np.array([0.0, 1.0]), 
               np.array([0.0, -2.0]), 0.1)
second_input = (0.01, 1.0, np.array([1.0, 0.3]), np.array([-1.0, -0.2]), np.array([0.0, 1.0]), 
               np.array([0.0, -2.0]), 0.1)
third_input = (0.5, 1.0, np.array([1.0, 0.3]), np.array([-1.0, -0.2]), np.array([0.0, 1.0]), 
               np.array([0.0, -2.0]), 0.1)

inputs = [convert_celestial_input(*inp) for inp in [first_input, second_input, third_input]]
ref_input = [{"m1" : value[0], "m2" : value[1], "r1" : value[2], "r2" : value[3], "v1" : value[4], "v2" : value[5], "dt" : value[6]} for value in inputs]

ref_output = [[np.array([3.834e+08, 8.640e+06]), np.array([-3.834e+08, -1.728e+07]), np.array([  -5.85699774, 1000.        ]), np.array([ 5.85699774e-02, -2.00000000e+03])],
[np.array([3.8340e+08, 1.2366e+08]), np.array([-3.834e+08, -9.396e+07]), np.array([ -5.34787994, 998.66303002]), np.array([ 5.34787994e-02, -1.99998663e+03])],
[np.array([3.8340e+08, 1.2366e+08]), np.array([-3.834e+08, -9.396e+07]), np.array([ -5.34787994, 998.66303002]), np.array([    2.67393997, -1999.33151501])]]

ex02_pb = ParametersBox(m1 = (0.01, 0.01, 10, 0.01, r'$m_1$ / $m_\mathrm{Moon}$'),
                            m2 = (1.0, 0.01, 10, 0.01, r'$m_2$ / $m_\mathrm{Earth}$'),
                            x1 = (1.0, -10, 10, 0.01, r'$x_1(0)$ / $d(\mathrm{earth-moon})$'),
                            y1 = (0.0, -10, 10, 0.01, r'$y_1(0)$ / $d(\mathrm{earth-moon})$'),
                            v1_x = (0.0, -2, 2, 0.01, r"$v_{1x}$ / km/s"), # r'$v_{x1}(0)$ / km/s'),
                            v1_y = (1.0, -2, 2, 0.01, r'$v_{1y}$ / km/s'),
                            dt = (0.01, 0.001, 1, 0.001, r'$dt$ / days', dict(readout_format='.3f')),
                            T = (100.0, 2, 1000, 1, r'total simulation / days'),
                            refresh_mode ="click")
with plt.ioff():
    fig_ax_ex02,_ = plt.subplots(1, 2, figsize=(10, 5))

ex02_pyplot_output = AnimationOutput(fig_ax_ex02)
    
anim_duration = 10
anim_fps = 10
def make_anim_euler_simulation(m1, m2, x1, y1, v1_x, v1_y, dt, T, code_input, visualizers):
    
    print_output = visualizers[0]
    print_output.clear_output()
    
    figure = visualizers[1].figure
    axes = figure.get_axes()
    updater = code_input.get_function_object()
    with print_output:
        animation = make_anim_simulation(updater, figure, axes, m1, m2, x1, y1, v1_x, v1_y, dt, T,
                                        duration=anim_duration, fps=anim_fps)    
    # clears progress bars
    print_output.clear_output()        
    visualizers[1].animation = animation
        
ex02_code_demo = CodeDemo(
            input_parameters_box=ex02_pb,
            code_input= ex02_wci,
            check_registry=check_registry,
            visualizers = [ClearedOutput(),ex02_pyplot_output],
            update_visualizers = make_anim_euler_simulation,
            merge_check_and_update_buttons = False)

check_registry.add_check(ex02_code_demo,
                         inputs_parameters=ref_input,
                         reference_outputs = ref_output,
                         equal= np.allclose)

answer_registry.register_answer_widget("ex02-function", ex02_code_demo)

In [ ]:
display(ex02_code_demo)

In [ ]:
# hack to avoid a mysterious figure disappearence bug
ex02_code_demo.visualizers[1].clear_output()

The widget above allows you to run the integration with different starting conditions and integration time step. Experiment with a trajectory initialized with earth-moon settings (set sliders to $(m_1,m_2,x_1,y_1,v_{1x},v_{1y})=$ (0.01,1.0,1.0,0.0,0.0,1.0)) and varying the integration time step.

<span style="color:blue">**02b** Is the total energy conserved? What is the largest time step you can use before the trajectory becomes completely unstable? </span>

_NB: you will see that for this problem the accuracy of the integrator degrades progressively as $dt$ is increased, so there isn't a clear-cut transition from a "stable" to "unstable" regime.  To give a consistent answer, look roughly for the value that leads to a doubling of the orbit radius after one revolution._

In [ ]:
ex2b_txt = TextareaAnswer("Enter your answer here")
answer_registry.register_answer_widget("ex2b-answer", ex2b_txt)
display(ex2b_txt)

<span style="color:blue">**02c** Try with a more eccentric trajectory (set sliders to $(m_1,m_2,x_1,y_1,v_{1x},v_{1y})=$ (0.01,1.0,1.0,0.0,-0.5,1.0). At which point of the orbit is the violation of energy conservation more severe? Is the stability limit the same as for the earth-moon settings? </span>

In [ ]:
ex2c_txt = TextareaAnswer("Enter your answer here")
answer_registry.register_answer_widget("ex2c-answer", ex2c_txt)
display(ex2c_txt)

The forward Euler integrator is not very performant, both in terms of theoretical accuracy, nor in terms of practical stability. Several [high-order integrators](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods) exist for generic differential equations. When integrating Hamilton's equations, there are considerations other than the asymptotic accuracy of the approximate integration. For example, Newtonian dynamics is time reversible, and [symplectic](https://en.wikipedia.org/wiki/Symplectic_integrator) (a property related to how a swarm of approximate trajectories started from a given volume of $\mathbf{x},\mathbf{p}$ space evolve in time). 

A simple integrator that preserves these two properties is the _velocity Verlet_ integrator,

$$
\mathbf{p}(dt/2) = \mathbf{p}(0) + \frac{dt}{2} \mathbf{f}(0)\\
\mathbf{x}(dt) = \mathbf{x}(0) + dt \mathbf{p}(dt/2)/m \\
\mathbf{p}(dt) = \mathbf{p}(dt/2) + \frac{dt}{2} \mathbf{f}(dt) \\
$$

In practice, the momentum is evolved in two steps, first using the force evaluated at the starting position, and then using the force evaluated at the final position. 

<span style="color:blue">**03a** Complete the function below to implement a velocity Verlet integrator for the equations of motion of two planets. </span>



In [ ]:
ex03_wci = WidgetCodeInput(
        function_name="verlet_update", 
        function_parameters="m1, m2, r1, r2, v1, v2, dt",
        docstring="""Update the position of the particles in a two-body problem using
        the Verlet integrator described above.
        Takes as arguments the masses of the two bodies (m1,m2), the initial positions
        and velocities (r1,r2,v1,v2) and the time step (dt). Returns the new positions
        and velocities (r1_new,r2_new,v1_new,v2_new). 
        >> All positions and velocities are numpy arrays. <<
""",
        function_body="""

import numpy as np
return r1, r2, v1, v2 # <-- remove this line after having finished the implementation of the function
G = 6.67430e-11 # gravitational constant in SI units
# you can compute forces by calling this function with the positions of the two bodies
def get_forces(r1, r2):
    ...
    return f1, f2

# Modify this to implement a velocity Verlet integrator. 
f1, f2 = get_forces(r1, r2)

v1_mid = ...
v2_mid = ...


r1_new = ...
r2_new = ...

f1, f2 = get_forces(r1_new, r2_new)

v1_new = ...
v2_new = ...

return r1_new, r2_new, v1_new, v2_new
"""
        )


def reference_verlet_update(m1, m2, r1, r2, v1, v2, dt):
    import numpy as np

    def get_forces(r1, r2):
        delta = r2 - r1
        norm = np.sqrt(np.sum(delta ** 2))
        direction = delta / norm

        f1 = direction * G*m1*m2 / (norm ** 2)
        f2 = -f1
        return f1, f2

    # edit these before updating the reference values...
    f1, f2 = 0,0# 

    v1_mid = 0
    v2_mid = 0


    r1_new = 0
    r2_new = 0

    f1, f2 = 0,0 #

    v1_new = 0
    v2_new = 0

    return r1_new, r2_new, v1_new, v2_new
    
first_input = (0.01, 1.0, np.array([1.0, 0.0]), np.array([-1.0, 0.0]), np.array([0.0, 1.0]), 
               np.array([0.0, -2.0]), 0.1)
second_input = (0.01, 1.0, np.array([1.0, 0.3]), np.array([-1.0, -0.2]), np.array([0.0, 1.0]), 
               np.array([0.0, -2.0]), 0.1)
third_input = (0.5, 1.0, np.array([1.0, 0.3]), np.array([-1.0, -0.2]), np.array([0.0, 1.0]), 
               np.array([0.0, -2.0]), 0.1)

inputs = [convert_celestial_input(*inp) for inp in [first_input, second_input, third_input]]
#ref_values = [[inp, reference_verlet_update(*inp)] for inp in inputs]
ref_values = [[np.array([3.83374698e+08, 8.64000000e+06]), np.array([-3.83399747e+08, -1.72800000e+07]), np.array([ -5.85218015, 999.90116804]), np.array([ 5.85218015e-02, -1.99999901e+03])],
              [np.array([3.83376897e+08, 1.23654224e+08]), np.array([-3.83399769e+08, -9.39599422e+07]), np.array([ -5.2813386 , 998.59152525]), np.array([ 5.28133860e-02, -1.99998592e+03])], 
              [np.array([3.83376897e+08, 1.23654224e+08]), np.array([-3.83388449e+08, -9.39571121e+07]), np.array([ -5.28141457, 998.59150239]), np.array([    2.64070728, -1999.29575119])]]
                   
ex03_pb = ParametersBox(m1 = (0.01, 0.01, 10, 0.01, r'$m_1$ / $m_\mathrm{Moon}$'),
                            m2 = (1.0, 0.01, 10, 0.01, r'$m_2$ / $m_\mathrm{Earth}$'),
                            x1 = (1.0, -10, 10, 0.01, r'$x_1(0)$ / $d(\mathrm{earth-moon})$'),
                            y1 = (0.0, -10, 10, 0.01, r'$y_1(0)$ / $d(\mathrm{earth-moon})$'),
                            v1_x = (0.0, -2, 2, 0.01, r"$v_{1x}$ / km/s"), # r'$v_{x1}(0)$ / km/s'),
                            v1_y = (1.0, -2, 2, 0.01, r'$v_{1y}$ / km/s'),
                            dt = (0.01, 0.001, 1, 0.001, r'$dt$ / days', dict(readout_format='.3f')),
                            T = (100.0, 2, 1000, 1, r'total simulation / days'),
                            refresh_mode ="click")

fig_ax_ex03,_ = plt.subplots(1, 2, figsize=(10, 5))
ex03_pyplot_output = AnimationOutput(fig_ax_ex03)

def make_anim_verlet_simulation(m1, m2, x1, y1, v1_x, v1_y, dt, T, code_input, visualizers):
    print_output = visualizers[0]
    figure = visualizers[1].figure
    axes = figure.get_axes()
    print_output.clear_output()
    updater = code_input.get_function_object()
    with print_output:
        animation = make_anim_simulation(updater, figure, axes, 
                                        m1, m2, x1, y1, v1_x, v1_y, dt, T,
                                        duration=anim_duration, fps=anim_fps)    
    print_output.clear_output()
    visualizers[1].animation = animation 
        
ex03_code_demo = CodeDemo(
            input_parameters_box=ex03_pb,
            code_input= ex03_wci,
            check_registry=check_registry,
            visualizers = [ClearedOutput(),ex03_pyplot_output],
            update_visualizers = make_anim_verlet_simulation,
            merge_check_and_update_buttons = False)

check_registry.add_check(ex03_code_demo,
                         inputs_parameters=ref_input,
                         reference_outputs = ref_values,
                         equal= np.allclose)

answer_registry.register_answer_widget("ex03-function", ex03_code_demo)

In [ ]:
display(ex03_code_demo)

In [ ]:
# needed to avoid a mysterious output disappearance bug
ex03_code_demo.visualizers[1].clear_output()

<span style="color:blue">**03b** Experiment with a trajectory initialized at earth-moon conditions and varying the integration time step. Try also the more eccentric trajectory (setting $v_{1x}=-0.5$ km/s). Find the stability limit and compare with the observations made for the forward Euler integrator. </span>

_NB: as for the previous exercise, use the time step that leads to a doubling of the orbit radius after one period as the instability limit._

In [ ]:
ex3b_txt = TextareaAnswer("Enter your answer here")
answer_registry.register_answer_widget("ex3b-answer", ex3b_txt)
display(ex3b_txt)

# Classical molecular dynamics (MD) for molecules and crystals

This far we have looked at the use of integrators to predict the trajectories of celestial bodies. As long as we treat nuclei as classical particles (which works decently unless one considers cryogenic temperatures, or light atoms [such as hydrogen](http://arxiv.org/abs/1803.00600)) the motion of atoms follows the same Hamiltonian equations, driven by the interatomic potentials we have learned about in the [dedicated module](./04-Potentials.ipynb). 

Then, we use integrators implemented in `ase.md` to run some short trajectories. The usage is pretty simple

```python
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution, Stationary
from ase.md.verlet import VelocityVerlet
from ase.calculators import eam
from ase import units

structure = ase.Atoms(...)
structure.calc = eam.EAM(...)

# set the initial velocities of the atoms from a Maxwell-Bolztmann distribution at 300K
MaxwellBoltzmannDistribution(structure, temperature_K=300)
# removes the center-of-mass velocity 
Stationary(structure)


# initialize the VV integrator
vv_integrator = VelocityVerlet(structure, 2 * units.fs)  # time step of 2fs

# run for a given number of steps
vv_integrator.run(1000)
```

You may note that we call `MaxwellBoltzmannDistribution` to set the initial velocities of the atoms. 

The problem with this strategy is that the simulation will run to completion, and we are only able of checking what is happening after it is `VelocityVerlet.run` returns (it will modify `structure` in place). In order to follow what's going on, we need to store copies of the structure and/or its properties after short trajectory segments. This is called an _MD loop_ and allows to analyze the outcome of the simulation by _post processing_.

```python
trajectory = []
for i in range(100):
    vv_integrator.run(10)
    print("V=%10.5f  K=%10.5f" %(structure.get_potential_energy(), structure.get_kinetic_energy) )    
    trajectory.append(structure.copy()) # <-- must make a copy otherwise the reference gets overwritten
```

_NB: the `"format_string" % (tuple_of_values)` syntax allows to format nicely the output. See [here](https://docs.python.org/3/library/stdtypes.html#old-string-formatting) for reference, or [here](https://docs.python.org/3/tutorial/inputoutput.html) for a more modern approach_

In [ ]:
ex04_wci = WidgetCodeInput(
        function_name="LJ_dimer_MD", 
        function_parameters="time_step",
        docstring="""
Performs Molecular Dynamics for the dimer described by Lennard-Jones potential. 
:param time_step: time step for molecular dynamic in fs. 
""",
        function_body="""
import numpy as np
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution, Stationary
from ase.md.verlet import VelocityVerlet
from ase.calculators import lj
from ase import units
import ase.io
import math
from tqdm.notebook import tqdm

# LJ parameters fitted to yield roughly equilibrium distance and binding energy of H2
e0 = 4.74 # in eV
d0 = 0.74 # in Å

structure = ase.Atoms("H2", positions=[[0,-d0/2,0], [0,d0/2,0]])
structure.calc = lj.LennardJones(sigma=d0/(2**(1/6)), epsilon=e0, rc = 4 * d0)

np.random.seed(123456)
MaxwellBoltzmannDistribution(structure, temperature_K=2000)
Stationary(structure)
vv_integrator = VelocityVerlet(structure, time_step * units.fs)  

filename = 'module_06-ar_trajectory.extxyz'
trajectory = []
structure.info['time'] = 0
structure.info['potential_energy'] = structure.get_potential_energy()
structure.info['kinetic_energy'] = structure.get_kinetic_energy()
structure.info['dimer_distance'] = structure.get_distance(0,1)
trajectory.append(structure.copy())
# run segments corresponding to 0.05 fs, and a total of 5fs
nstep = math.ceil(0.05 / time_step)
pbar = tqdm(range(int(5/(nstep*time_step))))
for i in pbar:
    vv_integrator.run(nstep)
    pbar.set_description(f"V={structure.get_potential_energy():10.5f} K={structure.get_kinetic_energy():10.5f}")
    structure.info['time'] = i*time_step*nstep    
    structure.info['potential_energy'] = structure.get_potential_energy()
    structure.info['kinetic_energy'] = structure.get_kinetic_energy()
    structure.info['dimer_distance'] = structure.get_distance(0,1)
    trajectory.append(structure.copy())

ase.io.write(filename, trajectory)
return filename
"""
        )

def visualise_md_lj(time_step, code_input, visualizers):    
    print_output = visualizers[0]
    with print_output:
        filename = code_input.get_function_object()(time_step)
    if filename is None:
        return
    
    frames = ase.io.read(filename, ':')
    times = [structure.info['time'] for structure in frames]
    potential_energies = [structure.info['potential_energy'] for structure in frames]
    kinetic_energies = [structure.info['kinetic_energy'] for structure in frames]
    dimer_distances = [structure.info['dimer_distance'] for structure in frames]
    for f in frames:
        f.info.clear()
        f.arrays.pop('momenta')
    
    properties=dict(
                time=times,
                potential_energy=potential_energies,
                kinetic_energy = kinetic_energies,
                total_energy = np.asarray(potential_energies)+np.asarray(kinetic_energies),
                dimer_distance = dimer_distances
                )

    chemiscope.write_input('module-06_H2_trajectory.chemiscope.json.gz', frames=frames, properties=properties)
    mycs = chemiscope.show(frames, properties,
                         settings={"structure":[{"bonds":False, "unitCell":False,"supercell":{"0":1,"1":1,"2":1},
                                                "keepOrientation": True, 'playbackDelay': 100}],
                                   'map': {
                                       'x' : {'property':'time' },
                                       'y' : {'property':'total_energy' },
                                       'color' : {'property':'dimer_distance' }
                                   } }
                                   )
    with print_output:
         display( mycs )

            
ex04_pb = ParametersBox(time_step=(0.05,0.01,0.5,0.01, r"time step, fs"),refresh_mode="click")
ex04_code_demo = CodeDemo(
            code_input= ex04_wci,
            input_parameters_box=ex04_pb,
            visualizers = [ClearedOutput()],
            update_visualizers = visualise_md_lj,
            merge_check_and_update_buttons = True)

answer_registry.register_answer_widget("ex04-function", ex04_code_demo)
display(ex04_code_demo)

[Download chemiscope datafile](./module-06_H2_trajectory.chemiscope.json.gz)

<span style="color:blue">**04a** The widget above integrates MD for an H₂ dimer. Look at the trajectory, plotting potential, kinetic and total energy. How do the three quantities change over time? Comment in particular on the scale of the fluctuations in kinetic and potential energy in comparison with the fluctuations in total energy. Is the behavior during compression and during stretching the same? Why? </span>

_Hint: to answer the last question, think of the shape of the LJ potential, and try to change the initialization temperature to lower and higher values._

In [ ]:
ex4a_txt = TextareaAnswer("Enter your answer here")
answer_registry.register_answer_widget("ex4a-answer", ex4a_txt)
display(ex4a_txt)

<span style="color:blue">**04b** Increase progressively the time step and re-run the simulation. What happens to the different energies? What is the largest time step you can use before the simulation becomes unstable?  </span>

_NB: similar to the planetary case, this is not a clear-cut transition, but ultimtely the molecule will break down. Give a rough estimate (+- 0.1fs) of the point where the error in the conservation of the total energy is comparable to the fluctuations in potential and kinetic energy_

In [ ]:
ex4b_txt = TextareaAnswer("Enter your answer here")
answer_registry.register_answer_widget("ex4b-answer", ex4b_txt)
display(ex4b_txt)

<span style="color:blue">**04c** Change the function to generate an He dimer, without changing the LJ parameters. In practice, you are just changing the masses to 4 atomic mass units. What do you observe in terms of the vibrational frequencies, and of the limiting time step for the dynamics? </span>

In [ ]:
ex4c_txt = TextareaAnswer("Enter your answer here")
answer_registry.register_answer_widget("ex4c-answer", ex4c_txt)
display(ex4c_txt)

<span style="color:blue">**05a** Now, a slightly more serious exercise. Write a function that generates a 2x2x2 supercell of _fcc_ aluminum, instrument the structure with an EAM calculator, initialize the velocities at 400K and runs 1ps of molecular dynamics. The time step in fs is given as a parameter.  </span>

_NB: most of the "logging" code is already implemented_

In [ ]:
ex05_wci = WidgetCodeInput(
        function_name="aluminum_MD", 
        function_parameters="time_step",
        docstring="""
Performs Molecular Dynamics for the 2x2x2 supercell of fcc aluminum described by EAM potential. 
:param time_step: time step for molecular dynamic in fs. 
""",
        function_body="""
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution, Stationary
from ase.md.verlet import VelocityVerlet
from ase.calculators import eam
from ase import units
import ase.io
import math
from tqdm.notebook import tqdm
filename = 'aluminum_trajectory.extxyz'
return filename # <-- remove this after having completed the code below
structure = ...

structure.calc = eam.EAM(potential='data/Al99.eam.alloy')

MaxwellBoltzmannDistribution(...)
Stationary(structure)
vv_integrator = VelocityVerlet(...)  


trajectory = []
structure.info['time'] = 0
structure.info['potential_energy'] = structure.get_potential_energy()
structure.info['kinetic_energy'] = structure.get_kinetic_energy()
trajectory.append(structure.copy())

# runs for 1ps storing a structure every 20fs (already set up)
nstep = math.ceil(20 / time_step)
pbar = tqdm(range(int(1000/(nstep*time_step))))
for i in pbar:  # this is the MD loop

    vv_integrator.run(nstep)
    
    # diagnostics
    pbar.set_description(f"V={structure.get_potential_energy():10.5f} K={structure.get_kinetic_energy():10.5f}")
    structure.info['time'] = i*time_step*nstep        
    structure.info['potential_energy'] = structure.get_potential_energy()
    structure.info['kinetic_energy'] = structure.get_kinetic_energy()
    trajectory.append(structure.copy())

# write out the trajectory 
ase.io.write(filename, trajectory)
return filename
"""
        )


ex05_pb = ParametersBox(time_step=(5.0,1,50,1, r"time step, fs"),refresh_mode="click")

def visualise_md_aluminum(time_step, code_input, visualizers):
    print_output = visualizers[0]
    with print_output:
        filename = code_input.get_function_object()(time_step)
    if filename is None:
        return
    
    frames = ase.io.read(filename, ':')
    
    times = [structure.info['time'] for structure in frames]
    potential_energies = [structure.info['potential_energy'] for structure in frames]
    kinetic_energies = [structure.info['kinetic_energy'] for structure in frames]
    for f in frames:
        f.info.clear()
        f.arrays.pop('momenta')
        
    properties=dict(
                time=times,
                potential_energy=potential_energies,
                kinetic_energy = kinetic_energies,
                total_energy = np.asarray(potential_energies)+np.asarray(kinetic_energies)
                )

    chemiscope.write_input("module_06-md_aluminum_nve.chemiscope.json.gz", frames=frames, properties=properties)
    with print_output:
        display(chemiscope.show(frames, properties,
                         settings={"structure":[{"bonds":False, "unitCell":True,"supercell":{"0":1,"1":1,"2":1},
                                   "environments": {"cutoff": 40}}],
                                   'map': {'y' : {'property':'total_energy' }} } )
               )


ex05_code_demo = CodeDemo(
            code_input= ex05_wci,
            input_parameters_box=ex05_pb,
            visualizers = [ClearedOutput()],
            update_visualizers = visualise_md_aluminum,
            merge_check_and_update_buttons = True)

answer_registry.register_answer_widget("ex05-function", ex05_code_demo)

In [ ]:
display(ex05_code_demo)

[Download chemiscope datafile](./module_06-md_aluminum_nve.chemiscope.json.gz)

<span style="color:blue">**05b** What is the largest time step you can use before the simulation becomes unstable? What are the differences relative to the hydrogen molecule above that explain the different behavior?  </span>

_NB: when you approach the stability limit some trajectories will blow up completely in the short time that is simulated by the demo, and some will not. You don't have to be very precise in quoting the point where things start go awry, just give a rough estimate to within a few fs._

In [ ]:
ex5b_txt = TextareaAnswer("Enter your answer here")
answer_registry.register_answer_widget("ex5b-answer", ex5b_txt)
display(ex5b_txt)

# Constant-temperature simulations

The simulations in the previous section describe the evolution of a material at a constant total energy $H=V(\mathbf{x}) + \mathbf{p}^2/2m$. In most practical situations, materials are not studied in isolation, but as part of a larger setup that is in thermal equilibrium with the environment. This is even more relevant because usually a simulation supercell is meant to describe a small portion of a macroscopic sample, and one must simulate the flow of energy between different parts of the system. 

There are dozens of schemes that have been proposed to modify Hamilton's equations to describe the energy exchange with a heat bath. Here we will use [Langevin dynamics](https://en.wikipedia.org/wiki/Langevin_equation), in which the time evolution for $\mathbf{p}$ is written as 

$$
\dot{\mathbf{p}} = -\frac{\partial V}{\partial\mathbf{x}} -\gamma \mathbf{p} + \sqrt{2m\gamma k_B T} \boldsymbol{\xi}
$$

$-\gamma \mathbf{p}$ corresponds to a viscous drag term, and $\sqrt{2m\gamma k_B T} \boldsymbol{\xi}$ describes stochastic collisions that compensate the energy dissipated by friction ($\boldsymbol{\xi}$ corresponds to uncorrelated Gaussian noise. The net result is that the energy fluctuates, in a way that is consistent with isothermal conditions at temperature $T$. 

A Langevin integrator is implemented in ASE in the `ase.md.langevin` module. Its usage is very similar to that for the NVE integrator

```python
from ase.md.langevin import Langevin
from ase import units

lan = Langevin(structure, timestep=2*units.fs, temperature_K=300, friction=1/(1000*units.fs))

lan.run(1000)
```

The friction is expressed in inverse time units, and $1/\gamma$ roughly indicates the time it takes for the momenta to equilibrate at the target temperature. Note that a too large friction would slow down diffusion, and so reduce the efficiency of the simulation. A value around $1/\text{ps}$ is usually a decent guess. 

<span style="color:blue">**06** Write a routine that runs constant-temperature molecular dynamics for a 2x2x2 supercell of Al. Use an aggressive thermostat time constant ($1/\gamma$) of 200fs, and a time step of 10fs. Run first 500fs to equilibrate the structure, and a further 2ps to accumulate statistics.  The function has already a stub of a MD loop that will save the trajectories that you run, for further processing. </span>

_NB_: once you get something that works, run at several temperatures, e.g. 100K, 200K, 400K, 700K - the resulting trajectories will be used further down. DON'T CHANGE THE NAMING CONVENTION. Each trajectory should take a few minutes, so you can start working on the next answer while you run these.

In [ ]:
ex06_wci = WidgetCodeInput(
        function_name="aluminum_langevin", 
        function_parameters="T",
        docstring="""
Performs Langevin Dynamics for the 2x2x2 supercell of fcc aluminum described by EAM potential. 
:param time_step: time step for molecular dynamic in fs. 
""",
        function_body="""
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution, Stationary
from ase.md.langevin import Langevin
from ase.calculators import eam
from ase import units
import ase.io
import math
from tqdm.notebook import tqdm
filename = f'aluminum_langevin_T_{T}.extxyz'

return filename # <-- remove this after having completed the code below

structure = ...

# initialize potential (don't forget to instrument the cell)
calc = eam.EAM(potential='data/Al99.eam.alloy')

# initialize the velocities at 400K, and remove the center of mass
MaxwellBoltzmannDistribution(...)
Stationary(structure)
lan = Langevin(...)  

print(f"Equilibrating at {T}K (be patient, it'll take ~1 minute)")
lan.run(50)


trajectory = []

structure.info['index'] = 0
structure.info['potential energy'] = structure.get_potential_energy()
structure.info['kinetic energy'] = structure.get_kinetic_energy()
trajectory.append(structure.copy())
pbar = tqdm(range(200))
for i in pbar:
    lan.run(1)  # save at every step
    pbar.set_description(f"V={structure.get_potential_energy():10.5f} K={structure.get_kinetic_energy():10.5f}")

    structure.info['index'] = i
    structure.info['potential energy'] = structure.get_potential_energy()
    structure.info['kinetic energy'] = structure.get_kinetic_energy()
    trajectory.append(structure.copy())

ase.io.write(filename, trajectory)
return filename
"""
        )

ex06_pb = ParametersBox(T = (100,10,1000,10,r'Target temp.'),refresh_mode = "click")
def langevin_updater(T, code_input, visualizers):
    print_output = visualizers[0]
    with print_output:
        filename = code_input.get_function_object()(T)
        frames = read(filename, "1:")
        properties = chemiscope.extract_properties(frames)
        mycs = chemiscope.show(frames, properties,
                     settings={"structure":[{"bonds":False, "unitCell":True,"supercell":{"0":1,"1":1,"2":1},
                                            "keepOrientation": True, 'playbackDelay': 100}],
                               'map': {
                                   'x' : {'property':'index' },
                                   'y' : {'property':'potential energy' }
                               } }
                               )
        display(mycs)

ex06_code_demo = CodeDemo(
            code_input= ex06_wci,
            input_parameters_box=ex06_pb,
            visualizers = [ClearedOutput()],
            update_visualizers = langevin_updater,
            merge_check_and_update_buttons = True)

answer_registry.register_answer_widget("ex06-function", ex06_code_demo)
display(ex06_code_demo)

Termal fluctuations cause vibrations of the atoms in a crystal around their equilibrium positions. These vibrations have all sorts of effects, from thermal expansion to changes of the electrical conductivity. One of the most direct consequences of the atomic motion is the attenuation of the intensity of diffraction peaks, quantified by a term known as the [Debye-Waller factor](https://en.wikipedia.org/wiki/Debye%E2%80%93Waller_factor). 

The central quantity is the _mean square displacement_ (MSD)

$$
\langle u^2 \rangle = \frac{1}{N} \langle \sum_i (\mathbf{x}_i - \mathbf{x}_i^{(0)})^2\rangle
$$

where $\mathbf{x}_i^{(0)}$ indicatees the equilibrium lattice position for the atom $i$, and $\mathbf{x}_i$ the position of the same atom in a snapshot of a finite-temperature simulation. $N$ is the number of atoms in the supercell.

$\langle \cdot \rangle$ indicates averaging over thermal fluctuations, but we can compute it (through the so-called [ergodic hypothesis](https://en.wikipedia.org/wiki/Ergodic_hypothesis)) running a finite-temperature molecular dynamics simulations, and collecting representative snapshots of the structure. In the previous demo we ran several of these finite-temperature trajectories, so now we can load them, and use them to compute the MSD at various temperatures. 

<span style="color:blue">**07** Write a routine that, given a pre-computed trajectory, evaluates the mean-square displacement of the atoms and returns its value. 
Compute the mean of the positions at all time steps along the trajectory as an estimate of the equilibrium reference position $\mathbf{x}^{(0)}_i$, so that the function can be self-contained, and does not require building the idealized minimum-energy structure. </span>

_Hints:_
* Note that the MSD is a scalar - this function should return a single value for each `filename` it loads
* You can achieve the appropriate averaging with explicit summations, but you can also more easily (and quickly) achieve the same goal with a judicious use of [`numpy.mean`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.mean.html)
* As you could show using statistical mechanics, for a harmonic crystal the MSD should grow linearly with $T$. Here you may observe small deviations due to anharmonicity, and noise due to the limited length of the simulations.

In [ ]:
ex07_wci = WidgetCodeInput(
        function_name="compute_msd", 
        function_parameters="filename",
        docstring="""
Computes the mean square displacement of the positions of the atoms over the trajectory. 
:param filename: path to the saved trajectory in the extxyz format. 
""",
        function_body="""

import ase.io
import numpy as np

structures = ase.io.read(filename, ':')
return 0.0 # <-- remove this after having completed the code below
return ...
"""
        )

def plot_msd(code_input,visualizers):
    ax = visualizers[1].figure.get_axes()[0]
    import os
    files = os.listdir('.')
    T_grid, msd_values = [], []
    for file in files:
        if file.startswith('aluminum_langevin_T_'):
            try:
                T = float(file.split('.')[0].split('_')[-1])
            except:
                pass
            func = code_input.get_function_object()
            msd = func(file)
            T_grid.append(T)
            msd_values.append(msd)
            
    T_grid, msd_values = np.array(T_grid), np.array(msd_values)
    indices = np.argsort(T_grid)
    T_grid = T_grid[indices]
    msd_values = msd_values[indices]
    
    ax.plot(T_grid, msd_values, 'o', color = 'blue')
    ax.plot(T_grid, msd_values, color = 'blue')
    ax.set_xlabel('$T$ / K')
    ax.set_ylabel('\n\n\nmsd/ Å$^2$')
                     
ex07_figure,_ = plt.subplots(1, figsize=(6,3.8), tight_layout=True)
ex07_pyplot_output = PyplotOutput(ex07_figure)
 
ex07_code_demo = CodeDemo(
            code_input= ex07_wci,
            check_registry=check_registry,
            visualizers = [ClearedOutput(),ex07_pyplot_output],
            update_visualizers = plot_msd,
            merge_check_and_update_buttons=False
           )

check_registry.add_check(ex07_code_demo,
                         inputs_parameters=[{"filename": "module_06-ex07_ref.xyz"}],
                         reference_outputs = [0.008466126604490386], 
                         equal=np.allclose
                        )     

answer_registry.register_answer_widget("ex07-function", ex07_code_demo)

display(ex07_code_demo)

# Melting point calculations

Now we run something a bit more complicated, in fact pushing the limits of what can be achieved with ASE and pure Python code. We will run a simulation in which Al is heated above its melting point, and then quenched back to low temperature. 

This can be achieved with a loop that adjusts the target temperature of the thermostat between a low value $T_{\mathrm{low}}$ and a high value $T_{\mathrm{high}}$, running in between short stretches of MD simulation.

In [ ]:
def plot_ramp(nramp, temp_lo, temp_hi,visualizers):
    ax = visualizers[1].figure.get_axes()[0]
    ax.plot(np.arange(nramp*2), 
            np.concatenate([
            np.linspace(temp_lo, temp_hi, nramp),
            np.linspace(temp_hi, temp_lo, nramp)
            ]) )
    ax.set_xlabel("MD segment")
    ax.set_ylabel(r'$T$/K')
    
ramp_pb = ParametersBox(nramp = (200,50,4000,50, r'$n_{\mathrm{ramp}}$'),
                       temp_lo = (350.,100,1000,50, r'$T_{\mathrm{low}}/K$'),
                       temp_hi = (3500.,1500,4000,500, r'$T_{\mathrm{high}}/K$'),
                      )

ramp_figure,_ = plt.subplots(1, figsize=(8,3.8), tight_layout=True)
ramp_pyplot_output = PyplotOutput(ramp_figure)

ramp_demo = CodeDemo(
            input_parameters_box=ramp_pb,
            visualizers = [ClearedOutput(),ramp_pyplot_output],
            update_visualizers = plot_ramp)
display(ramp_demo)
ramp_demo.run_demo()

The function below runs a melt-and-quench simulation. Try to understand it as much as you can: it uses several non-trivial algorithms, such as constant-pressure integrator that allow to change the size of the simulation cell in response to the internal pressure, which is needed to accomodate thermal expansion and the latent volume of fusion. It is also _slow_, because everything is implemented in Python: start by running with the default parameters, keeping in mind it might already take up to one hour to complete the simulation.
At the end of the simulation, the routine saves a `.xyz` file containing the simulation "movie". We will look at it further down. Later you may want to run more simulations, changing tre supercell size, the length of the ramp, and the temperature bounds: if you experiment with multiple parameters, change the `filename` so that you can load the trajectory later. 

In [ ]:
ex08_wci = WidgetCodeInput(
        function_name="melt_and_quench", 
        function_parameters="nrep, nramp, temp_lo, temp_hi",
        docstring="""
Creates a nrep×nrep×nrep Al fcc structure, and runs a MD simulation in which the temperature 
is raised from temp_low to temp_hi over nramp short trajectory segments. 
:return: name of the trajectory file
Structures to be visualized and line energies estimated for the structures along the relaxation
""",
        function_body="""
import numpy as np
from ase.io import write
from ase import Atoms
from ase.calculators import lj, eam
from ase.md.nptberendsen import NPTBerendsen
from ase.md.langevin import Langevin
from ase import units
from tqdm.notebook import tqdm

# use a LJ potential with parameters fitted to match lattice parameter and cohesive energy of Al. aggressive cutoff to reduce cost
calc = lj.LennardJones(sigma=2.62, epsilon=0.41, rc=2*2.62)
#calc = eam.EAM(potential='data/Al99.eam.alloy')   #<<- this is way too slow, but you can try it
a0 = 4.05 # lattice parameter of Al
# constructs a unit cell
fcc_cell = Atoms("Al4", cell=np.eye(3)*a0, positions=a0*np.asarray([[0,0,0],[0.5,0.5,0],[0.5,0,0.5],[0,0.5,0.5]] ),
                 pbc=True, calculator=calc ) 

# creates a supercell
suxcell = fcc_cell.repeat(nrep)
# attach calculator
suxcell.calc = calc

# this is a long story. ASE doesn't have a decent barostat, NPT is very unstable, and Berendsen 
# doesn't sample the correct ensemble. so, we use a combination of Berendsen with a loose coupling and 
# a Langevin integrator to enforce canonical sampling
dyn = NPTBerendsen(suxcell, timestep=2*units.fs, temperature_K=temp_lo, pressure_au=2*units.bar, 
                   taut=1e4*units.fs, taup=1e2*units.fs, compressibility_au=2.2 )
lan = Langevin(suxcell, timestep=2*units.fs, temperature_K=temp_lo, friction=0.02)

filename = "traj-exx.xyz"

# prints some stats
from time import time
start = [0.0]
iramp = [1]
traj = []

# ugly but effective: store reference to globals in the default args
def printenergy(atoms=suxcell, t=traj, md=dyn, start=start, nramp=nramp, iramp=iramp):  
    elapsed = time() - start[0]
    epot = atoms.get_potential_energy() / len(atoms)
    ekin = atoms.get_kinetic_energy() / len(atoms)
    pressure = np.trace(atoms.get_stress(voigt=False))/3
    a = atoms.copy(); a.calc = atoms.calc
    volume = np.linalg.det(a.cell)
    a.arrays.pop('momenta')
    a.info['time'] = md.dt*md.get_number_of_steps()/(1000*units.fs)
    a.info['target_temperature'] = dyn.temperature
    a.info['potential'] = epot
    a.info['kinetic_temperature'] = ekin / (1.5 * units.kB)
    a.info['stress'] = pressure
    a.info['volume'] = volume
    t.append(a)
    print('Energy/at.: V = %.3feV  K = %.3feV (T=%3.0fK)  '
          'Volume = %.3fÅ³, Time (elapsed/left): %.3fs/%.3fs' % (epot, ekin, ekin / (1.5 * units.kB), volume, elapsed, elapsed*(2*nramp/iramp[0]-1)))

start[0] = time()
lan.run(400)

# attaches the function as a callback - it'll be called every 50 steps to store and print the trajectory
dyn.attach(printenergy, interval=50)

# temperature change between ramp steps
framp = (temp_hi-temp_lo)*(1./nramp)
# ramp temperature up
print('Ramp')
for i in tqdm(range(nramp)):
    iramp[0]+=1
    lan.run(75)
    dyn.run(25)
    suxcell.wrap()
    lan.set_temperature(temperature_K=lan.temp/units.kB+framp)
    dyn.set_temperature(temperature_K=lan.temp/units.kB+framp)
# and then quench
print('Quench')
for i in tqdm(range(nramp)):
    iramp[0]+=1
    lan.run(75)    
    dyn.run(25)
    suxcell.wrap()
    lan.set_temperature(temperature_K=lan.temp/units.kB-framp)
    dyn.set_temperature(temperature_K=lan.temp/units.kB-framp)   

write(filename, traj)
return filename
"""
        )

In [ ]:
ex08_pb = ParametersBox(nrep = (2,1,3,1,r'$n_{\mathrm{rep}}$'), 
                       nramp = (100,50,4000,50, r'$n_{\mathrm{ramp}}$'),
                       temp_lo = (400.,100,1000,50, r'$T_{\mathrm{low}}/K$'),
                       temp_hi = (4000.,1500,5000,500, r'$T_{\mathrm{high}}/K$'),
                       refresh_mode="click"
                      )
def ex08_updater(nrep,nramp,temp_lo,temp_hi,code_input,visualizers):
    stdout = Output(layout=Layout(height='100%', max_height='200px'))
    vbox = VBox([stdout],layout=Layout(overflow='hidden'))
    display(vbox)
    with stdout:
        filename = code_input.get_function_object()(nrep,nramp,temp_lo,temp_hi)    

ex08_code_demo = CodeDemo(
            code_input= ex08_wci,
            input_parameters_box=ex08_pb,
            visualizers = [ClearedOutput()],
            update_visualizers = ex08_updater)

answer_registry.register_answer_widget("ex08-function", ex08_code_demo)
display(ex08_code_demo)

Here you can load the output trajectory and visualize it. Time is expressed in picoseconds ($10^{-12}$s), temperatures in K, volumes in Å³ and energies in eV/atom. 

In [ ]:
def load_traj(filename,visualizers):
    clear_output = visualizers[0]
    
    traj = read(filename, ":")
    for t in traj:
        t.arrays.pop('forces', -1)
        t.arrays.pop('momenta', -1)
        t.info.pop('stress', -1)
    properties = {'x': {'property' : 'target_temperature','target':'structure','values':[t.info["target_temperature"] for t in traj]}, 
                              'y': {'property' : 'potential','target':'structure','values':[t.info["potential"] for t in traj]},
                              'color':{'property': 'time','target':'structure','values':[t.info["time"] for t in traj]}}
    
    chemiscope.write_input("./module-06_temperature_ramp.chemiscope.json.gz", frames=traj,
                          properties=properties)
    clear_output.clear_output()  
    with clear_output:              
        cs = chemiscope.show(traj, properties=properties, 
                            settings={'structure': [{'bonds': False, 'unitCell': True, 'keepOrientation': True}]}
                            )
        display(cs)

load_traj_demo = CodeDemo(
            visualizers = [ClearedOutput()],
            input_parameters_box=ParametersBox(filename = Text("traj-exx.xyz"),refresh_mode = "click"),
            update_visualizers = load_traj)
display(load_traj_demo)

[Download chemiscope datafile](./module-06_temperature_ramp.chemiscope.json.gz)

Inspect the trajectory. Plot target temperature versus simulation time to visualize the temperature ramp. Then plot potential (a proxy for the enthalpy) versus time. It is also instructive to look at the potential energy as a function of the target temperature, which is the default visualization you get when you load a trajectory. 

<span style="color:blue">**08a** What happens as the temperature approaches the maximum temperature along the ramp? And as the temperature is decreased? Is the potential curve symmetric in the heating and cooling directions? Compare the starting and final configurations: what differences do you observe? </span>

In [ ]:
ex8a_txt = TextareaAnswer("Enter your answer here")
answer_registry.register_answer_widget("ex8a-answer", ex8a_txt)
display(ex8a_txt)

Besides the fact we are running the simulation with a very lousy model of the interatomic potential for Al (as we have seen in the [potentials module](./04-Potentials.ipynb)) there are two serious limitations to these simulations: they are too small (they suffer from _finite size effects_) and they are too fast (they are strongly out-of-equilibrium). Given that larger and longer simulations are too lengthy, we have prepared some for you to inspect. You can load a 100ps, 3×3×3 run inserting in the input box above the filename `data/traj-n3-r1000.xyz`, and an even longer, 400ps trajectory loading `data/traj-n3-r4000.xyz`.

<span style="color:blue">**08b** In these trajectories you can observe a clearer discontinuity in the potential (and the volume) as the temperature increases and decreases. Are these discontinuities at the same temperature? How can you explain this observation? What would be your best estimate for the melting point of Al with this model potential? </span>

In [ ]:
ex8b_txt = TextareaAnswer("Enter your answer here")
answer_registry.register_answer_widget("ex8b-answer", ex8b_txt)
display(ex8b_txt)

<span style="color:blue">**08c** Look carefully at the final configuration: can you give a better explanation for the difference in energy between the starting and the final states of the trajectory? Is this a completely unreasonable artefact? Can you think of realistic processing conditions that would lead to similar phenomena? </span>

In [ ]:
ex8c_txt = TextareaAnswer("Enter your answer here")
answer_registry.register_answer_widget("ex8c-answer", ex8c_txt)
display(ex8c_txt)